In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

In [3]:
d = salobj.Domain()

In [4]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [5]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

[None, None, None, None, None, None, None]

In [ ]:
await asyncio.sleep(5.)

In [8]:
await salobj.set_summary_state(atmcs, salobj.State.ENABLED, timeout=120)

RemoteEvent(ATAOS, 0, logMessage) falling behind; read 72 messages


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [12]:
await asyncio.sleep(1.)

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 100 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 66 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, elevationInPosition) falling behind; read 10 messages
AckCmdReader(ATPtg, 0, ackcmd) falling behind; read 12 messages
R

In [13]:
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling beh

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [7]:
await ataos.evt_heartbeat.next(flush=True)
await ataos.cmd_setLogLevel.set_start(level=logging.DEBUG, timeout=10)
# await salobj.set_summary_state(ataos, salobj.State.STANDBY)
await salobj.set_summary_state(ataos, salobj.State.ENABLED, settingsToApply="measured_20190908.yaml")

RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 30 messages
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 15 messages


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [10]:
await ataos.cmd_enableCorrection.set_start(m1=True, timeout=10)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 82 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 82 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 82 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 82 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 82 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 82 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 82 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 82 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 82 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 82 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 82 messages
RemoteTelemetry(ATPtg, 

In [ ]:
await ataos.cmd_disableCorrection.set_start(m1=True, timeout=10)

In [ ]:
await ataos.cmd_disable.start()

In [ ]:
await salobj.set_summary_state(atpne, salobj.State.ENABLED)

In [ ]:
attcs = ATTCS(atmcs=atmcs, 
              atptg=atptg, 
              ataos=ataos, 
              atpneumatics=atpne, 
              athexapod=athex, 
              atdome=atdome, 
              atdometrajectory=atdomtraj, 
              check={"atpneumatics": False, "athexapod": False, "atdome": True, "atdometrajectory": True})

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
ra = Angle('11:00:00', unit=u.hour)
dec = Angle('-22:26:41', unit=u.deg)
target_name="Test"

In [ ]:

radec = ICRS(ra, dec)

# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await attcs.slew(ra=radec.ra.hour, 
                 dec=radec.dec.deg,
                 rotPA=180.-alt_az.alt.deg,
                 rot_frame=ATPtg.RotFrame.FIXED,
                 rot_mode=ATPtg.RotMode.FIELD)

In [ ]:
await atptg.cmd_exitControl.start()

# Parking

In [ ]:
await asyncio.sleep(2.5)
atptg.evt_inPosition.flush()
await atptg.cmd_azElTarget.set_start(azDegs=0., elDegs=80.)
while True:
    inp = await atptg.evt_inPosition.next(flush=False, timeout=120.)
    if inp.inPosition:
        break

In [ ]:
await atptg.cmd_stopTracking.start()

In [ ]:
await salobj.set_summary_state(ataos, salobj.State.STANDBY)

In [ ]:
await asyncio.sleep(2.5)
ss = atptg.evt_summaryState.get()

In [ ]:
print(salobj.State(ss.summaryState))

In [ ]:
await asyncio.sleep(10.)

In [ ]:
await salobj.set_summary_state(atmcs, salobj.State.STANDBY, timeout=120)

In [ ]:
def mainValveCallback(data):
    print(data.state)

In [ ]:
atpne.evt_mainValveState.callback = mainValveCallback

In [ ]:
atpne.evt_mainValveState.callback = None

In [ ]:
await asyncio.sleep(10.)
# mv = atpne.evt_mainValveState.get()

In [ ]:
mv.state

In [ ]:
await atpne.cmd_closeMasterAirSupply.start(timeout=5)

In [ ]:
mv = await atpne.evt_mainValveState.next(flush=False, timeout=2)

In [ ]:
print(mv)

In [ ]:
mv.state, mv.private_sndStamp

In [ ]:
mv = atpne.evt_mainValveState.get()

In [ ]:
print(mv)

In [ ]:
mv.state, mv.private_sndStamp

In [ ]:
await atpne.cmd_openMasterAirSupply.start(timeout=5)

In [ ]:
mv = await atpne.evt_instrumentState.next(flush=False, timeout=2)

In [ ]:
mv.state

In [ ]:
await atpne.cmd_openInstrumentAirValve.start(timeout=5)

In [ ]:
try:
    await atpne.cmd_m1OpenAirValve.start(timeout=10)
except salobj.AckError as e:
    if e.ackcmd.ack == salobj.SalRetCode.CMD_NOPERM:
        print(e)
    else:
        raise e

In [ ]:
await asyncio.sleep(10.)

In [ ]:
await atdomtraj.cmd_start.start()

In [ ]:
await asyncio.sleep(1.)
atpne.evt_m1State.flush()
await atpne.cmd_m1OpenAirValve.start(timeout=10)
m1state = await atpne.evt_m1State.next(flush=False, timeout=10)
print(m1state.state)

In [ ]:
await asyncio.sleep(1.)
atpne.evt_m1State.flush()
await atpne.cmd_m1CloseAirValve.start(timeout=10)
m1state = await atpne.evt_m1State.next(flush=False, timeout=10)
print(m1state.state)

In [ ]:
await asyncio.sleep(5.)
atpne.evt_instrumentState.flush()
await atpne.cmd_openInstrumentAirValve.start(timeout=10)
m1state = await atpne.evt_instrumentState.next(flush=False, timeout=10)
print(m1state.state)
await asyncio.sleep(5.)

In [ ]:
await asyncio.sleep(5.)
atpne.evt_instrumentState.flush()
await atpne.cmd_closeInstrumentAirValve.start(timeout=10)
m1state = await atpne.evt_instrumentState.next(flush=False, timeout=10)
print(m1state.state)
await asyncio.sleep(5.)

In [ ]:
await atpne.cmd_openMasterAirSupply.start()